# Практическое задание к уроку 8. Рекуррентные нейронные сети RNN LSTM GRU
Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

построить свёрточные архитектуры

построить различные архитектуры с RNN

построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [151]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SimpleRNN, LSTM, GRU, Masking, GlobalMaxPool1D, Conv1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import f1_score, classification_report, roc_auc_score
import re

data = pd.read_excel('data/отзывы за лето.xls')
data = data[(data['Rating'] != 3)]
data['Rating'] = (data['Rating'] > 3).astype(int)
X_train, X_test, y_train, y_test = train_test_split(data['Content'], data['Rating'], test_size=0.33, random_state=42)

In [152]:
X_train.head()

2011                          Лучшее банковское приложение
20530                                   Хорошие приложения
10800    Не могу загрузить приложение пишет код ошибки ...
10372    Почему не работает приложение?! В чем проблема...
9141     Это ужас... Теперь ваше приложение абсолютно н...
Name: Content, dtype: object

In [153]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [154]:
X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [155]:
tokenizer = Tokenizer(num_words=None,
                      filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                      lower = False, split = ' ')
tokenizer.fit_on_texts(X_train)

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in X_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [156]:
def checkAccuracy():
    df = pd.DataFrame(columns=['Model type', 'Optimizer', 'Test score', 'Test accuracy'])
    model_types = ['SimpleRNN', 'LSTM', 'GRU']
    optimizers = ['Adam', 'Adagrad', 'RMSprop', 'SGD']
    for model_type in model_types:
        for optimizer in optimizers:
            model = Sequential()

            model.add(
                Embedding(input_dim=word_count,
                          input_length=training_length,
                          output_dim=30,
                          trainable=True,
                          mask_zero=True))
            model.add(Masking(mask_value=0.0))

            if model_type == 'SimpleRNN':
                model.add(SimpleRNN(64))
            elif model_type == 'LSTM':
                model.add(LSTM(64))
            elif model_type == 'GRU':
                model.add(GRU(64))

            model.add(Dense(64, activation='relu'))
            model.add(Dropout(0.5))
            model.add(Dense(1, activation='sigmoid'))

            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

            early_stopping=EarlyStopping(monitor='val_loss')


            history = model.fit(X_train, y_train,
                                batch_size=512,
                                epochs=10,
                                verbose=1,
                                validation_split=0.1,
                                callbacks=[early_stopping])

            results = model.predict(X_test, batch_size=512, verbose=1)
            results_tresh = (results > 0.5) * 1
            print('\n')
            print('Model type ' + model_type)
            print('Optimizer ' + optimizer)
            print(classification_report(y_test, results_tresh))

            score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
            new_row = {'Model type': model_type, 'Optimizer': optimizer, 'Test score': score[0], 'Test accuracy': score[1]}
            df = df.append(new_row, ignore_index=True)

    return df


In [157]:
df = checkAccuracy()

Epoch 1/10
13/13 [==============================] - 1s 32ms/step


Model type SimpleRNN
Optimizer Adam
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1029
           1       0.84      1.00      0.91      5488

    accuracy                           0.84      6517
   macro avg       0.42      0.50      0.46      6517
weighted avg       0.71      0.84      0.77      6517

 2/13 [===>..........................] - ETA: 0s - loss: 0.3815 - accuracy: 0.8467

C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


13/13 [==============================] - 0s 35ms/step - loss: 0.3857 - accuracy: 0.8421


C:\Users\tseyko\AppData\Local\Temp\ipykernel_21820\1104411923.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Epoch 1/10
13/13 [==============================] - 0s 23ms/step


Model type SimpleRNN
Optimizer Adagrad
              precision    recall  f1-score   support

           0       0.66      0.02      0.04      1029
           1       0.84      1.00      0.91      5488

    accuracy                           0.84      6517
   macro avg       0.75      0.51      0.48      6517
weighted avg       0.81      0.84      0.78      6517

13/13 [==============================] - 0s 28ms/step - loss: 0.6350 - accuracy: 0.8435
Epoch 1/10


...

C:\Users\tseyko\AppData\Local\Temp\ipykernel_21820\1104411923.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Epoch 1/10
13/13 [==============================] - 1s 13ms/step


Model type GRU
Optimizer SGD
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1029
           1       0.84      1.00      0.91      5488

    accuracy                           0.84      6517
   macro avg       0.42      0.50      0.46      6517
weighted avg       0.71      0.84      0.77      6517

 5/13 [==========>...................] - ETA: 0s - loss: 0.6169 - accuracy: 0.8434

C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


13/13 [==============================] - 0s 16ms/step - loss: 0.6173 - accuracy: 0.8421


C:\Users\tseyko\AppData\Local\Temp\ipykernel_21820\1104411923.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [158]:
print(df)

   Model type Optimizer Test score Test accuracy
0   SimpleRNN      Adam   0.385747      0.842105
1   SimpleRNN   Adagrad   0.634952      0.843486
2   SimpleRNN   RMSprop   0.332412      0.863588
3   SimpleRNN       SGD    0.46221      0.841645
4        LSTM      Adam    0.39894      0.842105
5        LSTM   Adagrad    0.68013      0.842105
6        LSTM   RMSprop   0.348547      0.842873
7        LSTM       SGD   0.632336      0.842105
8         GRU      Adam   0.375375      0.842105
9         GRU   Adagrad   0.674082      0.842105
10        GRU   RMSprop   0.332864      0.843486
11        GRU       SGD   0.617311      0.842105


In [159]:
def checkAccuracyCnn():
    df = pd.DataFrame(columns=['Model type', 'Optimizer', 'Test score', 'Test accuracy'])
    model_types = ['CNN', 'CNN + RNN', 'RNN + CNN']
    optimizers = ['Adam', 'Adagrad', 'RMSprop', 'SGD']
    for model_type in model_types:
        for optimizer in optimizers:
            model = Sequential()

            model.add(
                Embedding(input_dim=word_count,
                          input_length=training_length,
                          output_dim=30,
                          trainable=True,
                          mask_zero=True))
            model.add(Masking(mask_value=0.0))


            if model_type == 'CNN':
                model.add(Conv1D(128, 3, activation='relu'))
                model.add(GlobalMaxPool1D())
            elif model_type == 'CNN + RNN':
                model.add(Conv1D(128, 3, activation='relu'))
                model.add(SimpleRNN(64))
            elif model_type == 'RNN + CNN':
                model.add(SimpleRNN(64, return_sequences=True))
                model.add(Conv1D(128, 3, activation='relu'))
                model.add(GlobalMaxPool1D())

            model.add(Dense(64, activation='relu'))
            model.add(Dropout(0.5))
            model.add(Dense(1, activation='sigmoid'))

            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

            early_stopping=EarlyStopping(monitor='val_loss')


            history = model.fit(X_train, y_train,
                                batch_size=512,
                                epochs=10,
                                verbose=1,
                                validation_split=0.1,
                                callbacks=[early_stopping])

            results = model.predict(X_test, batch_size=512, verbose=1)
            results_tresh = (results > 0.5) * 1
            print('\n')
            print('Model type ' + model_type)
            print('Optimizer ' + optimizer)
            print(classification_report(y_test, results_tresh))

            score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
            new_row = {'Model type': model_type, 'Optimizer': optimizer, 'Test score': score[0], 'Test accuracy': score[1]}
            df = df.append(new_row, ignore_index=True)

    return df

In [160]:
dfCnn = checkAccuracyCnn()

Epoch 1/10
13/13 [==============================] - 0s 8ms/step


Model type CNN
Optimizer Adam
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1029
           1       0.84      1.00      0.91      5488

    accuracy                           0.84      6517
   macro avg       0.42      0.50      0.46      6517
weighted avg       0.71      0.84      0.77      6517

11/13 [========================>.....] - ETA: 0s - loss: 0.4305 - accuracy: 0.8436

C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


13/13 [==============================] - 0s 11ms/step - loss: 0.4326 - accuracy: 0.8421
Epoch 1/10


C:\Users\tseyko\AppData\Local\Temp\ipykernel_21820\2231848330.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


13/13 [==============================] - 0s 8ms/step


Model type CNN
Optimizer Adagrad
              precision    recall  f1-score   support

           0       0.73      0.10      0.17      1029
           1       0.85      0.99      0.92      5488

    accuracy                           0.85      6517
   macro avg       0.79      0.55      0.55      6517
weighted avg       0.83      0.85      0.80      6517

13/13 [==============================] - 0s 12ms/step - loss: 0.6801 - accuracy: 0.8518
Epoch 1/10


...

C:\Users\tseyko\AppData\Local\Temp\ipykernel_21820\2231848330.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


13/13 [==============================] - 1s 41ms/step


Model type RNN + CNN
Optimizer SGD
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1029
           1       0.84      1.00      0.91      5488

    accuracy                           0.84      6517
   macro avg       0.42      0.50      0.46      6517
weighted avg       0.71      0.84      0.77      6517

 1/13 [=>............................] - ETA: 1s - loss: 0.4296 - accuracy: 0.8516

C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tseyko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


13/13 [==============================] - 1s 64ms/step - loss: 0.4403 - accuracy: 0.8421


C:\Users\tseyko\AppData\Local\Temp\ipykernel_21820\2231848330.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [161]:
print(dfCnn)

   Model type Optimizer Test score Test accuracy
0         CNN      Adam   0.432625      0.842105
1         CNN   Adagrad   0.680108      0.851772
2         CNN   RMSprop   0.371437      0.842105
3         CNN       SGD   0.634326      0.842105
4   CNN + RNN      Adam   0.368132      0.845481
5   CNN + RNN   Adagrad   0.574492      0.842105
6   CNN + RNN   RMSprop   0.298982       0.87617
7   CNN + RNN       SGD   0.437571      0.842105
8   RNN + CNN      Adam   0.347426        0.8619
9   RNN + CNN   Adagrad   0.582349      0.842105
10  RNN + CNN   RMSprop   0.363683      0.851772
11  RNN + CNN       SGD   0.440253      0.842105


Старался делать максимально похожие сетки, что бы увидеть эффект именно от конкретного слоя. Для решения конкретно нашей задачи, лучше всего подошла сетка на сверточном слое. Рекуррентные сети обучались довольно долго, в сравнению с CNN. На мой взгляд RNN обучалась хуже изза коротких последовательностей, ведь твиты короткие, а после предобработки стали вообще крошечными.
